# Basemodel for summarization and question answering



In [ ]:
from transformers import pipeline
import pandas as pd

#1 Summarization



We are using the transformer **facebook/bart-large-cnn** https://huggingface.co/facebook/bart-large-cnn for **summarization**

First, we load our local data base with raw texts and convert it to a DataFrame.

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

df = pd.read_csv('/content/Copia de Contente Template - Database  .xlsx - Sheet1 (1).csv')


df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

,Text
0,Introduction to Vaccination: Vaccination is a ...
1,Human Papillomavirus (HPV) Definition: HPV is ...
2,Hepatitis A (HAV) Definition: Hepatitis A is a...
3,Hepatitis B (HBV) Definition: Hepatitis B is a...
4,"In conclusion, vaccination is a critical tool ..."


Checking how a single text looks like...

In [ ]:
texts = df['Text'].tolist()

texts[1]


"Human Papillomavirus (HPV) Definition: HPV is a group of viruses that affect the skin and mucous membranes, with some strains leading to genital warts and others causing cancers, particularly cervical cancer. Transmission: Primarily through skin-to-skin contact during sexual activity, including vaginal, anal, and oral sex. Symptoms: Often asymptomatic, but some strains cause genital warts, and others can lead to abnormal cell changes that may result in cancer. Vaccine Administration: The HPV vaccine is administered to protect against strains of the Human Papillomavirus that can lead to cervical cancer and other HPV-related cancers. Dosage: The vaccine is given as a series of injections: For those aged 9-14: Two doses, spaced 6 to 12 months apart. For those aged 15-26 (or those with certain immunocompromising conditions): Three doses, given over 6 months (0, 1-2, and 6 months). Administration: The vaccine is injected into the muscle, typically the upper arm or thigh. Effectiveness: The

Feeding the full dataframe to the model, that will create the summaries and make a new column with them.

Aditionally, we check if the text lenght is too long to avoid errors.

In [ ]:
# Check if any text exceeds the model's maximum length
for i, text in enumerate(texts):
  if len(text.split()) > 1024:
    print(f"Text at index {i} exceeds the maximum length of 1024 tokens.")

summaries = []
for text in texts:
    try:
        summary = summarizer(text, max_length=100, min_length=40, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    except IndexError:
        print(f"Skipping text due to IndexError: {text}")
        summaries.append("Summary not available due to text length.")

df['Summary'] = summaries

df[['Text', 'Summary']].head()

Your max_length is set to 100, but your input_length is only 65. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)
Your max_length is set to 100, but your input_length is only 48. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)
Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 100, but your input_length is only 98. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your

,Text,Summary
0,Introduction to Vaccination: Vaccination is a ...,Vaccines are crucial for preventing infectious...
1,Human Papillomavirus (HPV) Definition: HPV is ...,The HPV vaccine is administered to protect aga...
2,Hepatitis A (HAV) Definition: Hepatitis A is a...,Both males and females can get Hepatitis A. It...
3,Hepatitis B (HBV) Definition: Hepatitis B is a...,Both males and females can get Hepatitis B. It...
4,"In conclusion, vaccination is a critical tool ...",Vaccination is a critical tool in preventing s...


Downloading the new file with the summaries column.

In [ ]:
output_csv_file_path = '/content/output_summaries.csv'
df.to_csv(output_csv_file_path, index=False)


from google.colab import files
files.download(output_csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#2 Question Answering

In [ ]:
from transformers import pipeline
pipe = pipeline("question-answering", model="google-bert/bert-large-uncased-whole-word-masking-finetuned-squad")

# Texto de ejemplo sobre el cual se harán preguntas
context = """Injection.
The contraceptive injection contains the hormone progesterone and offers short-term prevention of pregnancy.
The contraceptive injection does not protect you from STIs, you should use a condom as well if you think you are at risk of an STI. May reduce heavy or painful periods. You don’t have to prepare for or interrupt sex. You don’t have to remember to take a pill everyday.
How To Use:
Works for up to 13 weeks (depending on type used) and is injected by a healthcare provider.
Side Effects:
It can take up to one year for your fertility and periods to return to normal after the injection wears off, so it may not be suitable if you want to have a baby in the near future.
Additional side effects can be; weight gain, headaches, mood swings, breast tenderness and irregular bleeding.
Good To Know:
The injection is not suitable for women who:
Are pregnant.
Want to become pregnant within the next year.
Want regular periods.
Have bleeding in between periods or after sex.
Are at risk of osteoporosis.
It may not be suitable for women who have or have had certain health conditions."""

# Pregunta de ejemplo
question = "is the injection safe"

# Realizar la pregunta usando el pipeline
result = pipe(question=question, context=context)

# Mostrar la respuesta
print(f"Question: {question}")
print(f"Answer: {result['answer']}")

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Question: is the injection safe
Answer: The injection is not suitable for women who:
Are pregnant
